Configurar  TASK

In [0]:
TASK = 'QQP'
splits = [
 f'{TASK}/train.tsv', 
 f'{TASK}/test.tsv',
 f'{TASK}/dev.tsv',
]
label_idx = [
             'question1',
             'question2',
            ]

Baixar TASK

In [0]:
!wget https://raw.githubusercontent.com/nyu-mll/jiant/master/scripts/download_glue_data.py
!python download_glue_data.py -d . -t $TASK

--2020-06-05 21:50:08--  https://raw.githubusercontent.com/nyu-mll/GLUE-baselines/master/download_glue_data.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 8004 (7.8K) [text/plain]
Saving to: ‘download_glue_data.py.3’

download_glue_data. 100%[===================>]   7.82K  --.-KB/s    in 0.001s  

2020-06-05 21:50:09 (6.18 MB/s) - ‘download_glue_data.py.3’ saved [8004/8004]

	Completed!


Translate

Prepare input

In [0]:
!pip install transformers --upgrade
!pip install mosestokenizer

# Translate sentences to English and save mapping dicitionary as json.
# Based on https://github.com/ruanchaves/assin/blob/master/sources/translate.py

import json
import math
import os
import torch

from tqdm.notebook import tqdm
from transformers import MarianMTModel, MarianTokenizer

model_name = 'Helsinki-NLP/opus-mt-en-ROMANCE'
tokenizer = MarianTokenizer.from_pretrained(model_name)
model = MarianMTModel.from_pretrained(model_name).to("cuda")

def decode(text):
    decoded = tokenizer.decode(
                    text, 
                    skip_special_tokens=True,
                    )

    return decoded

def needs_translation(sample, translations):
    needs = [text for text in sample if text not in translations.keys()]
    
    return needs

def translation(sample):
    src_txts = [f'>>pt_BR<< {text}' for text in sample]

    #translation_sample = tokenizer.prepare_translation_batch(src_txts)
    #translated = model.generate(translation_sample)

    model_inputs = tokenizer.prepare_translation_batch(src_txts).to("cuda")
    generated_ids = model.generate(
                                    model_inputs.input_ids, 
                                    attention_mask=model_inputs.attention_mask, 
                                    num_beams=2
                                    )
    
    translated = [decode(text) for text in generated_ids]

    return translated


def translate2dict(sentences, dictpath, batch_size):
    if not os.path.isfile(dictpath):
        with open(dictpath, 'w') as f:
            json.dump({}, f)

    with open(dictpath) as f:
        translations = json.load(f)
    
    remaining = needs_translation(sentences, translations)
    batch = math.ceil(len(remaining)/batch_size)

    for idx in tqdm(range(batch), "Translating"):
        keys = remaining[idx*batch_size:(idx+1)*batch_size] 
        values = translation(keys)
    
        new_translations = dict(zip(keys, values))
        translations.update(new_translations)
        torch.cuda.empty_cache()
        with open(dictpath, 'w+') as f:
            json.dump(translations, f)

from pprint import pprint
import pandas as pd
import os

sentences = list()
length = 0

for f in splits:
    table = pd.read_csv(f, sep = '\t', quoting=3, error_bad_lines=False)
    print(f) 
    #print(table.head())
    #print('\n'*3)
    
    for idx in label_idx:
        label = table[idx].copy()
        print(label)
        print('\n'*2)

        sentences.extend(list(label))
        length = length + label.size
        


assert length == len(sentences)
sentences = set(sentences)

     |████████████████████████████████| 675kB 466kB/s 
     |████████████████████████████████| 3.8MB 4.2MB/s 
     |████████████████████████████████| 1.1MB 12.2MB/s 
     |████████████████████████████████| 890kB 37.5MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893260 sha256=caa322bffa8a103e02496bb941958b47a99956242bf4dc02c3688cddc1de90e2
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses
  Created wheel for mosestokenizer: filename=mosestokenizer-1.1.0-cp36-none-any.whl size=49120 sha256=3c9b53e269ae61c6cf16490237e150a9c6ba684c3a9eaaf5c5166001d7dd81db
  Stored in directory: /root/.cache/pip/wheels/a2/e7/48/48d5e0f9c0cd5def2dfd7cb8543945f906448ed1313de24a29
  Created wheel for uctools: filename=uctools-1.2.1-cp36-none-any.whl size=3992 sha256=574349fd873a3dacc6be18d7f0e5e5ae06925db2025ae45569ed2919fffae759
  Stored in directory: /root/.cache/pip/wheels/f3/97/7

/usr/local/lib/python3.6/dist-packages/transformers/tokenization_utils.py:831: FutureWarning: Parameter max_len is deprecated and will be removed in a future release. Use model_max_length instead.
  category=FutureWarning,



QQP/train.tsv
0         How is the life of a math student? Could you d...
1                       How do I control my horny emotions?
2              What causes stool color to change to yellow?
3                               What can one do after MBBS?
4         Where can I find a power outlet for my laptop ...
                                ...                        
363865                 How do I make money flying my drone?
363866            What can you do with an economics degree?
363867         What type of current does a battery produce?
363868    Grammar: What is difference between schedule a...
363869    What is the easiest way to earn money using in...
Name: question1, Length: 363870, dtype: object



0         Which level of prepration is enough for the ex...
1                        How do you control your horniness?
2         What can cause stool to come out as little balls?
3                              What do i do after my MBBS ?
4         Would a second airport in

In [5]:
dictpath = f'{TASK}.json'
translate2dict(list(sentences),  dictpath, 100)

Replace with translation

In [0]:
from os import makedirs
import json

with open(dictpath) as reader:
    dictionary = json.load(reader)

translation_folder = f'translation/{TASK}'
makedirs(translation_folder)
for f in splits:
    table = pd.read_csv(f, sep = '\t', quoting=3, error_bad_lines=False)
    for idx in label_idx:
        table[idx] = table[idx].map(dictionary)
    table.to_csv(f'translation/{f}', quoting=3, index = None, sep = '\t')